# Quick Start

Generate binary forecasting questions from news articles in just a few steps.

> **Tip**: See [API.md](../API.md) for complete API reference and alternative configurations.

## Step 1: Install SDK

In [ ]:
%pip install lightningrod-ai

from IPython.display import clear_output
clear_output()

## Step 2: Import and Initialize Client

The `LightningRod` client provides access to transforms, datasets, files, and filesets. See [API.md](../API.md) for all available methods.

In [ ]:
import os
from lightningrod import LightningRod

api_key = os.getenv("LIGHTNINGROD_API_KEY", "your-api-key-here")
client = LightningRod(api_key=api_key)

## Step 3: Configure Seed Generator

Searches Google News for articles matching your query.

> **Alternatives**: Use `GdeltSeedGenerator` for large-scale datasets, or `FileSetSeedGenerator` for custom documents. See [API.md](../API.md) for details.

In [ ]:
from datetime import datetime, timedelta
from lightningrod import NewsSeedGenerator

seed_generator = NewsSeedGenerator(
    start_date=datetime.now() - timedelta(days=30),
    end_date=datetime.now(),
    interval_duration_days=7,
    search_query="technology announcements",
)

## Step 4: Configure Question Generator

Creates forecasting questions from the news articles.

> **Note**: We use `BINARY` answer type here. Other types: `CONTINUOUS` (numeric), `MULTIPLE_CHOICE`, `FREE_RESPONSE`. See examples 05-08.

In [ ]:
from lightningrod import QuestionGenerator, AnswerType, AnswerTypeEnum

answer_type = AnswerType(answer_type=AnswerTypeEnum.BINARY)

question_generator = QuestionGenerator(
    instructions="Generate forward-looking questions about technology announcements.",
    answer_type=answer_type,
)

## Step 5: Configure Labeler and Renderer

Labeler finds answers using web search. Renderer formats the final prompt.

> **Note**: The renderer populates the `prompt` field in each sample. See [API.md](../API.md) for sample structure.

In [ ]:
from lightningrod import WebSearchLabeler
from lightningrod._generated.models import QuestionRenderer

labeler = WebSearchLabeler(answer_type=answer_type)
renderer = QuestionRenderer(answer_type=answer_type)

## Step 6: Create Pipeline and Run

Combine all components and run to generate your dataset.

> **Alternative**: Use `client.transforms.submit()` to submit without waiting, then check status with `client.transforms.jobs.get(job_id)`. See [API.md](../API.md).

In [ ]:
from lightningrod import QuestionPipeline

pipeline_config = QuestionPipeline(
    seed_generator=seed_generator,
    question_generator=question_generator,
    labeler=labeler,
    renderer=renderer,
)

dataset = client.transforms.run(pipeline_config, max_questions=10)

## Step 7: View Results

Inspect the generated questions and answers. Each sample contains `seed`, `question`, `label`, `prompt`, and optional `context` and `meta` fields. See [API.md](../API.md) for the complete sample structure.

In [ ]:
print(f"Generated {dataset.num_rows} samples\n")

samples = dataset.to_samples()
for i, sample in enumerate(samples[:3]):
    print(f"Sample {i+1}:")
    if sample.question:
        print(f"  Question: {sample.question.question_text}")
    if sample.label:
        print(f"  Answer: {sample.label.label}")
        print(f"  Confidence: {sample.label.label_confidence:.2f}")
    print()